# Paso 1. Lectura del fichero

In [131]:
#Lectura del fichero

import pandas as pd
github_storage = "https://raw.githubusercontent.com/oeg-upm/Instituto-Estudios-Fiscales-ontologias/master/machine-learning/"

original_file = github_storage+"datos-IVA.csv"

# Leer csv donde se encuentran los datos
df = pd.read_csv(original_file, sep=";", engine='python', header=None, names=['persona.actividadEmpresarial','sujetoPasivo.domicilioFiscal','operacion.tipoOperacion','operacion.lugarRealizacion','operacion.exencion','factura.importe','factura.fecha','operacion.docs','exencion.hechoImponibleCubiertoEnExencion','factura.descripcion','sujetoPasivo.acogidoARegimenEspecial','numFacturasMismoTopico','infraccion','tipoInfraccion'])
df = df.drop([0]) #eliminado porque no se lee bien la primera línea
df = df.drop(columns=['exencion.hechoImponibleCubiertoEnExencion']) #eliminado pues no tiene valores relevantes
df


,persona.actividadEmpresarial,sujetoPasivo.domicilioFiscal,operacion.tipoOperacion,operacion.lugarRealizacion,operacion.exencion,factura.importe,factura.fecha,operacion.docs,factura.descripcion,sujetoPasivo.acogidoARegimenEspecial,numFacturasMismoTopico,infraccion,tipoInfraccion
1,venta de ropa,Toledo,adquisicion de bienes,Madrid,ExencionRecargoEquivalencia,140.0,22/7/21,Camisa de hombre de color blanco. Pantalón vaq...,Camisa de hombre de color blanco. Pantalón vaq...,RecargoEquivalencia,2.0,false,NaN
2,venta de calzado,Madrid,adquisicion de bienes,Madrid,ExencionRecargoEquivalencia,180.0,23/7/21,Mocasines de piel de color marrón,Mocasines de piel de color marrón,RecargoEquivalencia,3.0,false,NaN
3,venta de ropa,Toledo,adquisicion de bienes,Toledo,ExencionRecargoEquivalencia,230.0,24/7/21,Camisa de hombre de color azul. Pantalones de ...,Camisa de hombre de color azul. Pantalones de ...,RecargoEquivalencia,20.0,false,NaN
4,venta de calzado,Avila,adquisicion de bienes,Avila,ExencionRecargoEquivalencia,120.0,25/7/21,Zapatillas de deporte Adidas. Zapatos de cabal...,Zapatillas de deporte Adidas. Zapatos de cabal...,RecargoEquivalencia,15.0,false,NaN
5,venta de artículos de piel,Madrid,adquisicion de bienes,Toledo,ExencionRecargoEquivalencia,560.0,26/7/21,Abrigo de visón largo con mangas anchas,Abrigo de visón largo con mangas anchas,RecargoEquivalencia,2.0,true,infraccionEnAdquisicionDeBienes
6,joyería,Toledo,adquisicion de bienes,Madrid,ExencionRecargoEquivalencia,1200.0,27/7/21,Reloj de oro con incrustaciones de diamantes,Reloj de oro con incrustaciones de diamantes,RecargoEquivalencia,3.0,true,infraccionEnAdquisicionDeBienes
7,materiales de construcción,Avila,adquisicion de bienes,Avila,ExencionRecargoEquivalencia,3400.0,28/7/21,45 kg de cemento. 20 metros cuadrados de azule...,45 kg de cemento. 20 metros cuadrados de azule...,RecargoEquivalencia,1.0,true,infraccionEnAdquisicionDeBienes
8,joyería,Madrid,adquisicion de bienes,Madrid,ExencionRecargoEquivalencia,1800.0,29/7/21,Collar de perlas.,Collar de perlas.,RecargoEquivalencia,5.0,true,infraccionEnAdquisicionDeBienes
9,materiales de construcción,Madrid,prestación de servicios,Madrid,0,3500.0,30/7/21,45 kg de cemento. 20 metros cuadrados de azule...,45 kg de cemento. 20 metros cuadrados de azule...,0,20.0,false,NaN
10,obras y edificaciones,Madrid,prestación de servicios,Madrid,0,12000.0,31/7/21,Instalación de ventanas en habitaciones. Remod...,Instalación de ventanas en habitaciones. Remod...,0,3.0,false,NaN


# Paso 2. Tratamiento de lenguaje natural. Preprocesado de columnas con textos

In [132]:
# Librerías relacionadas con procesamiento de lenguaje natural
!python -m spacy download es_core_news_sm

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

import spacy
import es_core_news_sm

import codecs

     |████████████████████████████████| 16.2 MB 5.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [133]:
# Función de limpieza de tokens 
def clear_first_token(token):
  result = ""
  char = '\ufeff'
  for i in range(len(token)):
    if token[i] != char: result += token[i]
  return result

def preprocess(text):
  # Texto en minusculas
  text_lower = text.lower()
  # Tokenize 
  token_word = nltk.word_tokenize(text_lower, "spanish")
  # Algunos textos tienen caracteres raros al principio, por tanto hay que eliminarlos para que no influyan en el código 
  print(token_word)
  token_word[0] = clear_first_token(token_word[0])
  # Stopwords de palabras españolas
  stopword_spanish = stopwords.words("spanish")
  i = 0

  while(i < len(token_word)):
    # Se eliminan los tokens que se encuentren dentro de las stopwords 
    if token_word[i] in stopword_spanish:
      token_word.remove(token_word[i])
    #Se elimina cualquier token que sea distinto de caracteres alfanumericos
    elif not (token_word[i].isalpha()):
      token_word.remove(token_word[i])
    else:
      # Debido a que la libreria nltk no lematiza textos en español es necesario utilizar otra libreria de NLP la cual es Spacy
      word = sp(token_word[i])[0]
      token_word[i] = word.lemma_
      i = i+1
  return token_word

In [134]:
# Tokenización de las dos columnas com textos largos 
sp = es_core_news_sm.load()
df['operacion.docs.tokens']=df['operacion.docs'].apply(preprocess)
df['factura.descripcion.tokens']=df['factura.descripcion'].apply(preprocess)

['camisa', 'de', 'hombre', 'de', 'color', 'blanco', '.', 'pantalón', 'vaquero', 'de', 'talla', '42']
['mocasines', 'de', 'piel', 'de', 'color', 'marrón']
['camisa', 'de', 'hombre', 'de', 'color', 'azul', '.', 'pantalones', 'de', 'vestir', 'de', 'color', 'gris']
['zapatillas', 'de', 'deporte', 'adidas', '.', 'zapatos', 'de', 'caballero', 'de', 'talla', '43']
['abrigo', 'de', 'visón', 'largo', 'con', 'mangas', 'anchas']
['reloj', 'de', 'oro', 'con', 'incrustaciones', 'de', 'diamantes']
['45', 'kg', 'de', 'cemento', '.', '20', 'metros', 'cuadrados', 'de', 'azulejos', 'porcelanosa', '.', '30', 'sacos', 'de', 'yeso', '.', '2', 'kg', 'de', 'pegamento']
['collar', 'de', 'perlas', '.']
['45', 'kg', 'de', 'cemento', '.', '20', 'metros', 'cuadrados', 'de', 'azulejos', 'porcelanosa', '.', '30', 'sacos', 'de', 'yeso', '.', '2', 'kg', 'de', 'pegamento']
['instalación', 'de', 'ventanas', 'en', 'habitaciones', '.', 'remodelado', 'de', 'cocina', 'y', 'baño', 'principal']
['300kg', 'de', 'cemento', '.'

# Paso 3. Preparación del pipeline de procesamiento

In [135]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

In [136]:
# Function Transformer para el Feature Union
def get_numeric_data(x):
    return [record[:-2].astype(float) for record in x]

def get_text_data(x):
    return [record[-1] for record in x]

transfomer_numeric = FunctionTransformer(get_numeric_data)
transformer_text = FunctionTransformer(get_text_data)

In [161]:
# Features
numeric_features = ['factura.importe', 'numFacturasMismoTopico']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['persona.actividadEmpresarial', 'sujetoPasivo.domicilioFiscal', 'operacion.tipoOperacion', 'operacion.lugarRealizacion', 'operacion.exencion', 'sujetoPasivo.acogidoARegimenEspecial']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

textual_features = ['operacion.docs', 'factura.descripcion']
textual_transformer = Pipeline(steps=[
                ('vec', TfidfVectorizer(analyzer='word'))])

#factura.fecha
#infraccion
#tipoInfraccion
#operacion.docs.tokens 
#factura.descripcion.tokens 

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('text', textual_transformer, textual_features)])



# Pipeline para concatenar Vector Tfidf y datos numéricos
#pipeline = Pipeline([
#    ('features', FeatureUnion([
#            ('numeric_features', Pipeline([
#                ('selector', transfomer_numeric)
#            ])),
#             ('text_features', Pipeline([
#                ('selector', transformer_text),
#                ('vec', TfidfVectorizer(analyzer='word'))
#            ]))
#        ])),
#    ('clf', RandomForestClassifier())
#])


# Añadiendo el clasificador
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
 #                          ('features', FeatureUnion(['num','text'])),
                           ('clf', RandomForestClassifier())])


In [162]:
# Grid Search Parameters for RandomForest
param_grid = {'clf__n_estimators': np.linspace(1, 100, 10, dtype=int),
              'clf__min_samples_split': [3, 10],
              'clf__min_samples_leaf': [3],
              'clf__max_features': [7],
              'clf__max_depth': [None],
              'clf__criterion': ['gini'],
              'clf__bootstrap': [False]}
              
# Training config
kfold = StratifiedKFold(n_splits=7)
scoring = {'Accuracy': 'accuracy', 'F1': 'f1_macro'}
refit = 'F1'

# Perform GridSearch
rf_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kfold, scoring=scoring, 
                         refit=refit, n_jobs=-1, return_train_score=True, verbose=1)


X_Train, X_Test, Y_Train, Y_Test = model_selection.train_test_split(df.iloc[:,0:11],df['infraccion'],test_size=0.2, shuffle = True, random_state=0)

#X_Train = X_Train.iloc[:,0:11]
#X_train
#X_train = df.iloc[:,0:1]
#X_train = df['persona.actividadEmpresarial']
print(X_Train)
#Y_Train = Y_Train[:,0]
#print(Y_Train)
print(Y_Train)

datatypes = df.dtypes
print('Data type of each column of Dataframe :')
print(datatypes)
filteredColumns = df.dtypes[df.dtypes == np.object]
# list of columns whose data type is object i.e. string
listOfColumnNames = list(filteredColumns.index)
print(listOfColumnNames)


rf_model.fit(X_Train, Y_Train)
rf_best = rf_model.best_estimator_

   persona.actividadEmpresarial  ... numFacturasMismoTopico
2              venta de calzado  ...                    3.0
14        obras y edificaciones  ...                    3.0
24                venta de ropa  ...                   23.0
17         venta de electrónica  ...                    4.0
9    materiales de construcción  ...                   20.0
7    materiales de construcción  ...                    1.0
18                venta de ropa  ...                   25.0
5    venta de artículos de piel  ...                    2.0
3                 venta de ropa  ...                   20.0
6                       joyería  ...                    3.0
19             venta de calzado  ...                   30.0
10        obras y edificaciones  ...                    3.0
8                       joyería  ...                    5.0
20                      joyería  ...                   22.0
4              venta de calzado  ...                   15.0
1                 venta de ropa  ...    

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:    1.9s finished


ValueError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
import pickle 
import matplotlib.pyplot as plt
from collections import Counter

# Paso 4. Evaluación del modelo (no realizado aún)

In [ ]:
# Evaluacion del modelo 
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("Accuracy del modelo -> ",accuracy_score(predictions_SVM, Test_Y)*100)

print("Porcentaje de acierto para cada una de las clases: ")
print(classification_report(Test_Y, predictions_SVM))

print("Matriz de confusion test: ")
plt.clf()
fig,ax = plt.subplots(figsize=(12,12))
plot_confusion_matrix(SVM, Test_X_Tfidf, Test_Y, cmap=plt.cm.Blues, ax=ax, values_format='d')
plt.show() 

print("Matriz de confusion train: ")
plt.clf()
fig,ax = plt.subplots(figsize=(12,12))
plot_confusion_matrix(SVM, Train_X_Tfidf, Train_Y, cmap=plt.cm.Blues, ax=ax, values_format='d')
plt.show() 


In [ ]:
# Algoritmo SVM utilizando cross-validation
from sklearn.model_selection import cross_val_score

Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(df_procesado['Tokens'])

X = Tfidf_vect.transform(df_procesado['Tokens'])

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto', random_state=0)
scores = cross_val_score(SVM, X, df_procesado['ODS'], cv=10)

for index, value in enumerate(scores):
  print(f"Accuracy del fold {index} -> {value:.2%}")

print(f"Accuracy medio: {scores.mean():.2%}, desviación estandar: {scores.std():.2%}")

In [ ]:
# Variables X e Y para entrenar y probar el algoritmo SVM
# Training = 100%
Train_X = df_procesado['Tokens']
Train_Y = df_procesado['ODS']

# TF-IDF
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(df_procesado['Tokens'])

Train_X_Tfidf = Tfidf_vect.transform(Train_X)